### Hi,
Since using dash require connecting to a server,adjusting the input continously will sometime corrupt the process.  
Thus,i prefer creating dashboard directly in Jupyter Notebook 

##### Explanation

###### **1**-Import all the required libs  
###### **2**-Load the data  
  Here is an explanation of each line of code:

**spacex_df = pd.read_csv("spacex_launch_dash.csv"):**  
This line reads the CSV file "spacex_launch_dash.csv" and loads it into a pandas DataFrame named spacex_df.

**max_payload = spacex_df['Payload Mass (kg)'].max():**  
This line finds the maximum value of the 'Payload Mass (kg)' column in spacex_df and assigns it to the max_payload variable.

**min_payload = spacex_df['Payload Mass (kg)'].min():**  
This line finds the minimum value of the 'Payload Mass (kg)' column in spacex_df and assigns it to the min_payload variable.

**sites = spacex_df['Launch Site'].unique().tolist():**  
This line creates a list of unique launch sites in the 'Launch Site' column of spacex_df and assigns it to the sites variable.

**sites.insert(0, 'All Sites'):**  
This line inserts the string 'All Sites' at the beginning of the sites list using the insert() method. This is done to add an option to select all launch sites in the dashboard.


###### **3**-Define the widgets  
  Here is an explanation of each line of code:  

**site_dropdown**  
A dropdown widget with options set to sites, a default value of 'All Sites', and a description of 'Launch Site'. The style parameter is used to set the width of the description to 'initial'.  

**payload_slider**  
An IntRangeSlider widget with minimum and maximum values set to min_payload and max_payload, step size of 100, default values of [min_payload, max_payload], a description of 'Payload Range (kg)', and width of the description set to 'initial'.  

**pie_chart**  
A pie chart visualization created using the px.pie() function from the Plotly Express library. The data used for the pie chart is spacex_df, with values set to the 'class' column and names set to the 'Launch Site' column. The title of the pie chart is 'Total Success Launches by Site'.  

**scatter_plot**  
A scatter plot visualization created using the px.scatter() function from the Plotly Express library. The x-axis data is set to 'Payload Mass (kg)', the y-axis data is set to 'class', and the color of the points is set based on 'Booster Version Category'. The title of the scatter plot is 'Correlation between Payload and Success for different Booster Versions', and the width of the scatter plot is set to 960.

###### **4**-Define the callback functions  
  Here is an explanation of each line of code:  
        
**update_pie_chart(site)**  
Takes a site parameter and filters the spacex_df dataframe by the selected site.  
Then it counts the number of successful and unsuccessful launches for the filtered  
data and creates a new pie chart using px.pie() with the count data. Finally, it returns the new figure.  

**update_scatter_plot(payload)**  
Takes a payload parameter which is a tuple of minimum and maximum payload values selected  
using the Payload Range slider. It filters the spacex_df dataframe by the selected payload range and  
creates a new scatter plot using px.scatter() with filtered data.The title of the scatter plot is updated  
to include the selected payload range. Finally, it returns the new figure.  

**site_dropdown_event_handler(change)**  
The event handler for the Launch Site dropdown widget.  
It calls the update_pie_chart(site) function with the newly selected value and displays the new pie chart  
using clear_output() and display() functions.  

**payload_slider_event_handler(change)**  
The event handler for the Payload Range slider widget.  
It calls the update_scatter_plot(payload) function with the newly selected value and displays the new scatter plot  
using clear_output() and display() functions.


###### **5**-Bind the widgets with the event handlers  
  Here is an explanation of each line of code:  
        
**site_dropdown_event_handler**  
Is called whenever the value of the site_dropdown widget changes  

**payload_slider_event_handler**  
Is called whenever the value of the payload_slider widget changes.  

**observe()**  
Used to set up these event listeners  

**names**  
A parameter specifying which property of the widget to observe for changes (in this case, the 'value' property).

###### **6**-Display the widgets and initial figures  

Ok let's go  

###### **Since GitHub cannot render plotly,run in your local machine / google collab / kaggle

In [14]:
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display, clear_output

# Load the data

spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
sites = spacex_df['Launch Site'].unique().tolist()
sites.insert(0, 'All Sites')

#**********************************************************************************************************************
# Define the widgets

site_dropdown = widgets.Dropdown(options=sites, value='All Sites', 
                                 description='Launch Site:', 
                                 style={'description_width': 'initial'})

payload_slider = widgets.IntRangeSlider(min=min_payload, 
                                        max=max_payload, 
                                        step=100, 
                                        value=[min_payload, max_payload], 
                                        description='Payload Range (kg):', 
                                        style={'description_width': 'initial'})

pie_chart = px.pie(spacex_df, 
                   values='class', 
                   names='Launch Site', 
                   title='Total Success Launches by Site')

scatter_plot = px.scatter(spacex_df, 
                          x='Payload Mass (kg)', 
                          y='class', 
                          color='Booster Version Category', 
                          title='Correlation between Payload and Success for different Booster Versions',
                          width=960)


#**********************************************************************************************************************

# Define the callback functions

def update_pie_chart(site):
    filtered_df = spacex_df.copy()
    if site != 'All Sites':
        filtered_df = filtered_df[filtered_df['Launch Site'] == site]
    data = filtered_df['class'].value_counts()
    fig = px.pie(values=data.values, names=data.index, title=f"Total Success Launches for {site}")
    return fig

def update_scatter_plot(payload):
    filtered_df = spacex_df.copy()
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= payload[0]) & (filtered_df['Payload Mass (kg)'] <= payload[1])]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f"Correlation between Payload and Success for different Booster Versions (Payload Range: {payload[0]}-{payload[1]} kg)")
    return fig

# Define the event handlers
def site_dropdown_event_handler(change):
    fig = update_pie_chart(change.new)
    with output_pie_chart:
        clear_output(wait=True)
        display(fig)

def payload_slider_event_handler(change):
    fig = update_scatter_plot(change.new)
    with output_scatter_plot:
        clear_output(wait=True)
        display(fig)

#**********************************************************************************************************************

# Bind the widgets with the event handlers

site_dropdown.observe(site_dropdown_event_handler, names='value')
payload_slider.observe(payload_slider_event_handler, names='value')

#**********************************************************************************************************************

# Display the widgets and initial figures

display(widgets.HBox([site_dropdown, payload_slider]))
output_pie_chart = widgets.Output()
output_scatter_plot = widgets.Output()
display(output_pie_chart)
display(output_scatter_plot)
with output_pie_chart:
    display(pie_chart)
with output_scatter_plot:
    display(scatter_plot)
    
#**********************************************************************************************************************

Output()

Output()